In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy

# Veriyi oku
df = pd.read_csv("apple_quality.csv")

# NaN değerleri say
df.isna().sum()

# Veri hakkında bilgi al
df.info()

# NaN değerleri içeren satırları sil
df.dropna(inplace=True)

# "A_id" sütununu kaldır
df.drop(columns="A_id", inplace=True)

# Float olan sütunları int64 türüne dönüştür
float_labels = ["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness"]
for i in float_labels:
    df[i] = df[i].astype('int64')

# "Acidity" sütununu int64 türüne dönüştür, hata göz ardı edilecek
df["Acidity"] = pd.to_numeric(df["Acidity"], errors='ignore')

# "Quality" sütunundaki "good" ve "bad" değerlerini 1 ve 0'a dönüştür
df["Quality"] = df["Quality"].replace({"good": 1, "bad": 0})

# Veri setini özellikler ve hedef değişken olarak ayır
X = df.drop(columns="Quality", axis=1)
y = df["Quality"]

# Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=16)

# Model oluştur
model = Sequential([
    Input(shape=X_train.shape[1]),
    Dense(170, activation="relu"),
    Dropout(0.1),
    Dense(120, activation="relu"),
    Dense(50, activation="relu"),
    Dropout(0.2),
    Dense(30, activation="relu"),
    Dense(2, activation="sigmoid")
])

# Model özetini görüntüle
model.summary()

# Modeli derle
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])

# Modelin kontrol noktası ve erken durdurma için geri çağrılar
model_checkpoint = ModelCheckpoint("model.h5", monitor="val_loss", save_best_only=True, save_weights_only=False)
early_stopping = EarlyStopping(monitor="val_loss", min_delta=0.2, patience=20)
callbacks = [model_checkpoint, early_stopping]

# Modeli eğit
model.fit(X_train, y_train, batch_size=128, epochs=70, callbacks=callbacks, validation_split=0.2)


In [ ]:
df.info()

In [ ]:
print(X_test,y_test)

In [ ]:
from keras.models import load_model

model = load_model("model.h5")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import numpy as np

In [ ]:
tahmin_ortalama = np.mean(y_pred)

In [ ]:
apple=[]

for i in y_pred:
    if i > tahmin_ortalama:
        apple.append(1)
    else:
        apple.append(0)

In [ ]:
apple

In [ ]:
y_test